# Development: Score composite map/product
Generally scoring has been a matter of applying the model to samples with known LULC, and then comparing predicted to actual LULC. This approach does not work for directly scoring maps, such as a composite/mode map, because they do not represent a direct model output. This notebook demonstrates scoring an actual map, based on a DL product.

Generally, this notebook assumes that the target product is a 6-category areal map, which can also 
  
Date: 2019-02-13  
Author: Peter Kerins  

### Import statements
(may be over-inclusive)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#
import os
import sys
import shapely
import cartopy
import numpy as np

get_ipython().magic(u'matplotlib inline')
import matplotlib.pyplot as plt
import pickle

import descarteslabs as dl
print (dl.places.find('illinois')) ## TEST

ULU_REPO = os.environ["ULU_REPO"]
if not ULU_REPO in sys.path:
    sys.path.append(ULU_REPO+'/utils')
    sys.path.append(ULU_REPO)
print (sys.path)

import util_vectors
import util_rasters
import util_scoring
import util_workflow

## Preparation

### Set key variables

In [ ]:
# choose city
place = 'hyderabad'

In [ ]:
product='wri:ulu-india'
product_year='2019'
model_id = product

In [ ]:
# core
data_root='/data/phase_iv/'
data_path = data_root+place+'/'

resolution = 5  # Lx:15 S2:10

# tiling
tile_resolution = resolution
tile_size = 256
tile_pad = 32
tile_side = tile_size+tile_pad+tile_pad

In [ ]:
v_only = True

In [ ]:
# set features of original model used for mapping
# not used in scoring, just for record-keeping
s2_bands=['blue','green','red','nir','swir1','swir2','alpha']; suffix='BGRNS1S2A'  # S2, Lx
look_window=17

# # ground truth source: aue, aue+osm, aue+osm2
gt_type = 'aue'
gt_lot = '0'

stack_label, feature_count = util_workflow.build_stack_label(
        bands_vir=s2_bands[:-1],
        bands_sar=None,
        bands_ndvi=None,
        bands_ndbi=None,
        bands_osm=None,)

In [ ]:
# product-specific stuff
if product_year=='2019':
    start_datetime='2018-01-01'
    end_datetime='2021-01-01'
elif product_year=='2016':
    start_datetime='2015-01-01'
    end_datetime='2018-01-01'

In [ ]:
categories = [0,1,2,3,4,5]
categories_map = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
categories_remapping = [0,1,2]
categories_remapping_map = {0: 0, 1: 1, 2: 2, 3: 2, 4: 2, 5: 2}

### Load and inspect study area

In [ ]:
place_title = place.title()
place_shapefile = data_path+place_title+"_studyAreaEPSG4326.shp"

shape = util_vectors.load_shape(place_shapefile)
polygon = shape['geometry']['coordinates']
place_bbox = shape['bbox']

# using Albers projection
lonlat_crs = cartopy.crs.PlateCarree()
clat, clon = (place_bbox[0]+place_bbox[2])/2.0, (place_bbox[1]+place_bbox[3])/2.0
print ("center co-ordinates", clat, clon)
albers = cartopy.crs.AlbersEqualArea(central_latitude=clat, central_longitude=clon)

# visualize Study Region
fig = plt.figure(figsize=(6,6))
ax = plt.subplot(projection=albers) # Specify projection of the map here
shp = shapely.geometry.shape(shape['geometry'])
ax.add_geometries([shp], lonlat_crs)
ax.set_extent((place_bbox[0], place_bbox[2], place_bbox[1], place_bbox[3]), crs=lonlat_crs)
ax.gridlines(crs=lonlat_crs)
plt.show()

### Generate tiles

In [ ]:
tiles = dl.raster.dltiles_from_shape(tile_resolution, tile_size, tile_pad, shape)
single_tile_id = 22
highlights = {single_tile_id:'green'}
util_vectors.draw_tiled_area(shape, tiles, albers, lonlat_crs, highlights=highlights)

## Compare map and ground-truth 

Create matching lists (ie the nth element in one list refers to the same pixel as the nth element in the other) of actual and predicted LULC, based on the rasterized ground-truth and the composite/mode map.  

Simultaneously score using a full typology and the "standard" 3-category reduced typology. Obviously this is only applicable in the case of an areal (ie not roads) model.

In [ ]:
Yhat_full = []
Y_full = []

Yhat_remapping = []
Y_remapping = []

# load up locale lists if appropriate
if v_only:
    place_locales_filename = data_root+'models/'+'locales_'+place+'.pkl'
    with open(place_locales_filename, "rb") as f:
        place_locales = pickle.load(f,encoding='latin1')
    t_locales, v_locales = place_locales[place][0], place_locales[place][1]
    print(type(v_locales), v_locales)

for tile_id in range(len(tiles['features'])):
    if tile_id % 100 == 0:
        print('progress at tile #',tile_id)
    # get ground-truth tile
    gt_tile_path = (data_path+'gt'+'/'+place+'_'+gt_type+gt_lot+'_'+ 
                    str(tile_resolution)+'m'+'_'+'p'+str(tile_pad)+'_'+
                    'tile'+str(tile_id).zfill(4)+'_'+'lulc'+'.tif')
    tile_gt, geo, prj, cols, rows = util_rasters.load_geotiff(gt_tile_path,dtype='uint8')
    if tile_gt.shape != (tile_side, tile_side):
        raise Exception("bad gt tile shape:",tile_gt.shape)
    
    Y_img = tile_gt[tile_pad:-tile_pad,tile_pad:-tile_pad]
    # count non-no-data pixels (255), excluding padding, and skip those without gt
    n_pixels = np.sum(Y_img!=255) 
    if n_pixels == 0:
        continue
    
    print('processing non-empty gt tile: #',tile_id)
    print('categories present: ',np.unique(Y_img))
    
    # load locale tile if appropriate
    if v_only:
        locale_tile_path = (data_path+'gt'+'/'+place+'_'+gt_type+gt_lot+'_'+ 
                        str(tile_resolution)+'m'+'_'+'p'+str(tile_pad)+'_'+
                        'tile'+str(tile_id).zfill(4)+'_'+'locale'+'.tif')
        tile_locale, _,_,_,_ = util_rasters.load_geotiff(locale_tile_path,dtype='uint8')
        if tile_locale.shape != (tile_side, tile_side):
            raise Exception("bad locale tile shape:",tile_locale.shape)
        tile_locale_unpadded = tile_locale[tile_pad:-tile_pad,tile_pad:-tile_pad]
        
        locale_mask = np.isin(tile_locale_unpadded, v_locales)
        if np.sum(locale_mask) == 0:
            # if no pixels in a validation locale, skip to next tile
            print('skip training-only gt tile: #',tile_id)
            continue
            
        # blank out gt values for non-validation locales
        print (np.sum(Y_img!=255), Y_img)
        Y_img[~locale_mask] = 255
        print (np.sum(Y_img!=255), Y_img)
    
    tile = tiles['features'][tile_id]
    aoi=dl.scenes.DLTile.from_key(tile['properties']['key'])
#     print(tile)
    scenes, ctx = dl.scenes.search(aoi,products=[product],start_datetime=start_datetime,end_datetime=end_datetime)
    
#     print(type(ctx))
#     print(ctx)
#     ctx._resolution=tile_resolution
#     print(ctx, type(ctx))
    
#     print(scenes)
#     for scene in scenes:
#         print(scene)
    try:
        tile_lulc = scenes.mosaic(['lulc'], ctx, 
               mask_nodata=True, mask_alpha=None, bands_axis=0, resampler='near', 
               processing_level=None, scaling=None, data_type='Byte', raster_info=False)
    except ValueError as e: 
        print('skipping tile with empty SceneCollection!')
        print(e)
        continue
    
    if tile_lulc.shape != (1,tile_side, tile_side):
        raise Exception("bad lulc tile shape:",tile_lulc.shape)
    
    Yhat_img = tile_lulc[0,tile_pad:-tile_pad,tile_pad:-tile_pad]
    
    Yhat, Y = util_scoring.extract_scoring_arrays(Yhat_img, Y_img, categories, remapping=None)
    Yhat_alt, Y_alt = util_scoring.extract_scoring_arrays(Yhat_img, Y_img, categories_remapping, remapping='standard')
#     print(Yhat[0:20])
#     print(Yhat_alt[0:20])

    # this is where we test for an empty product
    # ie where map of urban india excluded portion of aue study area
    if (np.ma.is_masked(Yhat)) and len(np.unique(Yhat))==1:
        print ('skipping unmapped tile!')
        print (tile)
        continue
        
#     print(Yhat.shape, Y.shape)
    confusion = util_scoring.calc_confusion(Yhat, Y, categories)
    Yhat_full.extend(Yhat)
    Y_full.extend(Y)
    
    confusion = util_scoring.calc_confusion(Yhat_alt, Y_alt, categories_remapping)
    Yhat_remapping.extend(Yhat_alt)
    Y_remapping.extend(Y_alt)
    
    print()
    

#### Generate confusion matrix and statistics for entire city using the full, 6-category results

In [ ]:
confusion = util_scoring.calc_confusion(np.asarray(Yhat_full), np.asarray(Y_full), categories)

In [ ]:
recalls, precisions, accuracy = util_scoring.calc_confusion_details(confusion)
print (recalls)
print (precisions)
print (accuracy)

In [ ]:
# Calculate f-score
beta = 2
f_scores = (beta**2 + 1) * precisions * recalls / ( (beta**2 * precisions) + recalls )
f_score_average = np.mean(f_scores)
print (f_scores)
print (f_score_average)

#### Regenerate confusion matrix and statistics for entire city using the reduced, 3-category typology

In [ ]:
confusion_remapping = util_scoring.calc_confusion(np.asarray(Yhat_remapping), np.asarray(Y_remapping), categories_remapping) 

In [ ]:
recalls_remapping, precisions_remapping, accuracy_remapping = util_scoring.calc_confusion_details(confusion_remapping)
print (recalls_remapping)
print (precisions_remapping)
print (accuracy_remapping)

In [ ]:
# Calculate f-score
beta = 2
f_scores_remapping = (beta**2 + 1) * precisions_remapping * recalls_remapping / ( (beta**2 * precisions_remapping) + recalls_remapping )
f_score_average_remapping = np.mean(f_scores_remapping)
print (f_scores_remapping)
print (f_score_average_remapping)

## Record results

#### Log results for *full* typology

In [ ]:
if v_only:
    notes = 'validation ' + product_year + ' full typology '+'composite map scoring of '+place
else:
    notes = product_year + ' full typology '+'composite map scoring of '+place

In [ ]:
# expanding lists to match expected model_record stuff
recalls_expanded = [None,None,None,None,None,None,None,]
precisions_expanded = [None,None,None,None,None,None,None,]
f_scores_expanded = [None,None,None,None,None,None,None,]
for r in range(0,len(recalls)):
    recalls_expanded[r] = recalls[r]
    precisions_expanded[r] = precisions[r]
    f_scores_expanded[r] = f_scores[r]

In [ ]:
util_scoring.record_model_application(
        model_id, notes, place+'('+product_year+')', gt_type + gt_lot, resolution, stack_label, feature_count, 
        look_window, categories_map, 
        confusion, recalls_expanded, precisions_expanded, accuracy,
        f_scores_expanded, f_score_average)

#### Log results for *reduced* typology

In [ ]:
if v_only:
    notes_remapping = 'validation ' + product_year + ' reduced typology '+'composite map scoring of '+place
else:
    notes_remapping = product_year + ' reduced typology '+'composite map scoring of '+place

In [ ]:
# expanding lists to match expected model_record stuff
recalls_remapping_expanded = [None,None,None,None,None,None,None,]
precisions_remapping_expanded = [None,None,None,None,None,None,None,]
f_scores_remapping_expanded = [None,None,None,None,None,None,None,]
for r in range(0,len(recalls_remapping)):
    recalls_remapping_expanded[r] = recalls_remapping[r]
    precisions_remapping_expanded[r] = precisions_remapping[r]
    f_scores_remapping_expanded[r] = f_scores_remapping[r]

In [ ]:
util_scoring.record_model_application(
        model_id, notes_remapping, place+'('+product_year+')', gt_type + gt_lot, resolution, stack_label, feature_count, 
        look_window, categories_remapping_map, 
        confusion_remapping, recalls_remapping_expanded, precisions_remapping_expanded, accuracy_remapping,
        f_scores_remapping_expanded, f_score_average_remapping)

-------------------------